This cell is for running NFSP on Leduc and FHP

In [ ]:
import pyspiel
from cfr_utils import NFSPWrapper, nfsptrain
import copy
import os
import open_spiel
import numpy as np
import tensorflow as tf
import open_spiel.python
import open_spiel.python.algorithms
import open_spiel.python.algorithms.nfsp
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.disable_v2_behavior()
fhp = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 4, "numRanks":13, "numHoleCards": 2, "numBoardCards": "0 3", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
leduc = pyspiel.load_game("universal_poker", {"numPlayers":2, "numSuits": 2, "numRanks":3, "numHoleCards": 1, "numBoardCards": "0 1", "bettingAbstraction": "fcpa", "numRounds":2, "blind": "50 100"})
leducconfig = {"state_representation_size": 16}
fhpconfig = {"state_representation_size": 108}
leducgame = NFSPWrapper(leduc)
fhpgame = NFSPWrapper(fhp)
tf.config.set_visible_devices([], 'GPU')
num_players = 2
max_nodes = 10000000


In [ ]:
nodes = 0
games = [fhpgame,leducgame]
for i in games:
    i.reset()
    if i == leducgame:
        game_string = "leduc"
    else:
        game_string = "fhp"
    with tf.compat.v1.Session() as sess:
            agents = [open_spiel.python.algorithms.nfsp.NFSP(
            session=sess,
            player_id=_,
            state_representation_size=leducconfig["state_representation_size"] if i == leducgame else fhpconfig["state_representation_size"],
            num_actions=4,
            hidden_layers_sizes=[1024,512,1024,512],
            reservoir_buffer_capacity=30000000,
            anticipatory_param=0.1,
            batch_size=256, rl_learning_rate=0.1, sl_learning_rate=0.01, min_buffer_size_to_learn=1000, learn_every=256, optimizer_str="sgd", replay_buffer_capacity= 600000, epsilon_start=0.08, epsilon_end=0,
            ) for _ in range(num_players)]
            sess.run(tf.compat.v1.global_variables_initializer())
            print(agents)
            print(i)
            nfsptrain(agents, i,max_nodes, game_string)
            print("Finished training for game: ", game_string)
